In [34]:
import pandas as pd
import numpy as np
import datetime
from tiingo import TiingoClient
import indicoio

In [2]:
df = pd.read_csv("AAPL-daily.csv")

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-05-28,178.919998,180.589996,177.910004,178.229996,178.229996,27948200
1,2019-05-29,176.419998,179.350006,176.000000,177.380005,177.380005,28481200
2,2019-05-30,177.949997,179.229996,176.669998,178.300003,178.300003,21218400
3,2019-05-31,176.229996,177.990005,174.990005,175.070007,175.070007,27043600
4,2019-06-03,175.600006,177.919998,170.270004,173.300003,173.300003,40396100


In [7]:
def fix_dates(df):
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
    df["Start"] = df["Date"].apply(str)
    # now need to calculate the end
    six_days = lambda start_date: start_date + datetime.timedelta(days=6)
    df["End"] = df["Date"].apply(six_days).apply(str)
    df = df.drop(columns=["Date"])
    remove_time = lambda dt: dt[0:10]
    df["Start"] = df["Start"].apply(remove_time)
    df["End"] = df["End"].apply(remove_time)
    return df

In [8]:
df = fix_dates(df)
df.head()

,Open,High,Low,Close,Adj Close,Volume,Start,End
0,178.919998,180.589996,177.910004,178.229996,178.229996,27948200,2019-05-28,2019-06-03
1,176.419998,179.350006,176.000000,177.380005,177.380005,28481200,2019-05-29,2019-06-04
2,177.949997,179.229996,176.669998,178.300003,178.300003,21218400,2019-05-30,2019-06-05
3,176.229996,177.990005,174.990005,175.070007,175.070007,27043600,2019-05-31,2019-06-06
4,175.600006,177.919998,170.270004,173.300003,173.300003,40396100,2019-06-03,2019-06-09


In [14]:
client = TiingoClient({"api_key": "a265fc4a1013923f970d16e7348195074e97fcb0"})
def query_ticker(ticker, start, end):
    articles = client.get_news(tickers=[ticker], startDate=start, endDate=end)
    return articles

In [17]:
df["JSON"] = df.apply(lambda d: query_ticker("AAPL", d["Start"], d["End"]), axis=1)

In [27]:
def aggregate_jsons(json_list):
    result = []
    for json in json_list:
        headline = json["title"]
        desc = json["description"]
        x = headline + " " + desc
        result.append(x)
    return result

In [28]:
df["corpus"] = df["JSON"].apply(aggregate_jsons)

In [29]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,Start,End,JSON,corpus
0,178.919998,180.589996,177.910004,178.229996,178.229996,27948200,2019-05-28,2019-06-03,"[{'crawlDate': '2019-06-02T23:13:36.518022Z', ...",[4 Reasons Roku Stock Could Still Climb Higher...
1,176.419998,179.350006,176.000000,177.380005,177.380005,28481200,2019-05-29,2019-06-04,"[{'crawlDate': '2019-06-04T00:31:39.451597Z', ...",[Apple touts privacy in web sign-on war with F...
2,177.949997,179.229996,176.669998,178.300003,178.300003,21218400,2019-05-30,2019-06-05,"[{'crawlDate': '2019-06-05T01:51:44.603810Z', ...","[As Apple Kicks Off Developer Conference, Deve..."
3,176.229996,177.990005,174.990005,175.070007,175.070007,27043600,2019-05-31,2019-06-06,"[{'crawlDate': '2019-06-05T23:58:26.596237Z', ...",[The Bulls Break Loose On June 5 And There May...
4,175.600006,177.919998,170.270004,173.300003,173.300003,40396100,2019-06-03,2019-06-09,"[{'crawlDate': '2019-06-08T21:09:50.586665Z', ...",[A Fintech CEO Explains Why Gift Cards Are Mor...


In [35]:
indicoio.config.api_key = "026012f2d0b869994cf77bf084d58a97"

In [38]:
df["sentiment"] = df["corpus"].apply(indicoio.sentiment).apply(sum)

In [39]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,Start,End,JSON,corpus,sentiment
0,178.919998,180.589996,177.910004,178.229996,178.229996,27948200,2019-05-28,2019-06-03,"[{'crawlDate': '2019-06-02T23:13:36.518022Z', ...",[4 Reasons Roku Stock Could Still Climb Higher...,55.237436
1,176.419998,179.350006,176.000000,177.380005,177.380005,28481200,2019-05-29,2019-06-04,"[{'crawlDate': '2019-06-04T00:31:39.451597Z', ...",[Apple touts privacy in web sign-on war with F...,52.824310
2,177.949997,179.229996,176.669998,178.300003,178.300003,21218400,2019-05-30,2019-06-05,"[{'crawlDate': '2019-06-05T01:51:44.603810Z', ...","[As Apple Kicks Off Developer Conference, Deve...",47.648247
3,176.229996,177.990005,174.990005,175.070007,175.070007,27043600,2019-05-31,2019-06-06,"[{'crawlDate': '2019-06-05T23:58:26.596237Z', ...",[The Bulls Break Loose On June 5 And There May...,55.188811
4,175.600006,177.919998,170.270004,173.300003,173.300003,40396100,2019-06-03,2019-06-09,"[{'crawlDate': '2019-06-08T21:09:50.586665Z', ...",[A Fintech CEO Explains Why Gift Cards Are Mor...,53.120082
